# 29. 国旗画像のURLを取得する

テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [ ]:
# 問題29: 国旗画像のURLを取得する

import os
import re
import requests

# イギリスの記事ファイルのパス
data_dir = "../data"
uk_article_file = os.path.join(data_dir, "uk_article.txt")

# 基礎情報テンプレートのフィールド名と値を抽出する関数（問題25の関数）
def extract_basic_info_template(text):
    """記事から基礎情報テンプレートのフィールド名と値を抽出する関数"""
    # 基礎情報テンプレートの抽出
    template_pattern = r"\{\{基礎情報[^|]*?\|([\s\S]*?)\}\}"
    template_match = re.search(template_pattern, text)
    
    if not template_match:
        return {}
    
    template_content = template_match.group(1)
    
    # フィールドの抽出（行ごとに処理）
    basic_info = {}
    field_name = None
    field_value = []
    
    for line in template_content.split("\n"):
        # 新しいフィールドの開始
        field_match = re.match(r"\|\s*([^=]+?)\s*=\s*(.*)", line)
        if field_match:
            # 前のフィールドがあれば保存
            if field_name is not None:
                basic_info[field_name] = "\n".join(field_value).strip()
            
            # 新しいフィールドの開始
            field_name = field_match.group(1).strip()
            field_value = [field_match.group(2).strip()]
        else:
            # 現在のフィールドの値の続き
            if field_name is not None:
                field_value.append(line.strip())
    
    # 最後のフィールドを保存
    if field_name is not None:
        basic_info[field_name] = "\n".join(field_value).strip()
    
    return basic_info

In [ ]:
# 国旗画像のファイル名を抽出する関数
def extract_flag_filename(basic_info):
    """基礎情報テンプレートから国旗画像のファイル名を抽出する関数"""
    # 国旗画像のフィールド名（「国旗画像」または「旗画像」）
    flag_field_names = ["国旗画像", "旗画像"]
    
    # 国旗画像のフィールドを探す
    flag_filename = None
    for field_name in flag_field_names:
        if field_name in basic_info:
            flag_filename = basic_info[field_name]
            break
    
    return flag_filename

# MediaWiki APIを使用して画像のURLを取得する関数
def get_image_url(filename):
    """MediaWiki APIを使用して画像のURLを取得する関数"""
    # ファイル名の先頭に「ファイル:」または「File:」がない場合は追加
    if not filename.startswith(("ファイル:", "File:")):
        filename = "File:" + filename
    
    # MediaWiki APIのエンドポイント
    api_url = "https://ja.wikipedia.org/w/api.php"
    
    # APIリクエストのパラメータ
    params = {
        "action": "query",
        "format": "json",
        "prop": "imageinfo",
        "titles": filename,
        "iiprop": "url"
    }
    
    try:
        # APIリクエストを送信
        response = requests.get(api_url, params=params)
        data = response.json()
        
        # レスポンスから画像のURLを抽出
        pages = data["query"]["pages"]
        for page_id in pages:
            if "imageinfo" in pages[page_id]:
                return pages[page_id]["imageinfo"][0]["url"]
        
        return None
    except Exception as e:
        print(f"APIリクエストでエラーが発生しました: {e}")
        return None

In [ ]:
# メイン処理
try:
    # イギリスの記事を読み込む
    if os.path.exists(uk_article_file):
        with open(uk_article_file, "r", encoding="utf-8") as f:
            uk_article = f.read()
        
        # 基礎情報テンプレートのフィールド名と値を抽出
        basic_info = extract_basic_info_template(uk_article)
        
        # 国旗画像のファイル名を抽出
        flag_filename = extract_flag_filename(basic_info)
        
        if flag_filename:
            print(f"国旗画像のファイル名: {flag_filename}")
            
            # 国旗画像のURLを取得
            flag_url = get_image_url(flag_filename)
            
            if flag_url:
                print(f"国旗画像のURL: {flag_url}")
            else:
                print("国旗画像のURLを取得できませんでした。")
        else:
            print("国旗画像のファイル名を抽出できませんでした。")
    else:
        print(f"イギリスの記事ファイルが見つかりません: {uk_article_file}")
        print("問題20を先に実行して、イギリスの記事を抽出してください。")
        
except Exception as e:
    print(f"エラーが発生しました: {e}")

## 解説

この問題では、基礎情報テンプレートから国旗画像のファイル名を抽出し、MediaWiki APIを使用してその画像のURLを取得する方法を学びます。

### MediaWiki APIとは

MediaWiki APIは、WikipediaなどのMediaWikiベースのウェブサイトのコンテンツにプログラムからアクセスするためのインターフェースです。このAPIを使用することで、記事の内容、画像情報、編集履歴など、さまざまな情報を取得できます。

### 国旗画像のURLを取得する手順

1. **基礎情報テンプレートから国旗画像のファイル名を抽出する**
   - 基礎情報テンプレートには、「国旗画像」または「旗画像」というフィールドに国旗の画像ファイル名が記載されています。

2. **MediaWiki APIを使用して画像のURLを取得する**
   - `action=query`と`prop=imageinfo`を使用して、画像の情報を取得します。
   - `iiprop=url`パラメータを指定することで、画像のURLを取得できます。

### 実装のポイント

1. **APIリクエストの構築**
   - APIエンドポイント（`https://ja.wikipedia.org/w/api.php`）に対して、適切なパラメータを指定してリクエストを送信します。

2. **レスポンスの解析**
   - APIからのレスポンスはJSON形式で返されるため、`json()`メソッドを使用して解析します。
   - 画像情報は`pages`オブジェクト内の`imageinfo`配列に含まれています。

3. **エラー処理**
   - APIリクエストが失敗した場合や、画像情報が取得できなかった場合のエラー処理を実装します。

### 注意点

1. **ファイル名の形式**
   - Wikipediaのファイル名は「ファイル:」または「File:」で始まる必要があります。抽出したファイル名にこのプレフィックスがない場合は追加する必要があります。

2. **APIの利用制限**
   - MediaWiki APIには利用制限があるため、大量のリクエストを短時間に送信すると制限される可能性があります。

3. **画像の著作権**
   - Wikipediaの画像には著作権があるため、利用する際は適切なライセンス表示が必要です。